# Clustering
Autor: Piotr Branewski

## Założenia i cel
1. Załóżmy, że jesteśmy w stanie zaetykietować x% (np.. 5%) zbioru danych określając ich przynależność do K klas. Zbiór danych proszę wybrać dowolny (najlepiej jednak zrobić to na 2 zbiorach danych.
2. CEL: Zaetykietuj resztę obiektów elementów danych wykorzystując metody klasteryzacji


## Zadanie - szczegóły
1. Poprzez klasteryzację zbioru na C klastrów „zgadnij” jak największą liczbę etykiet. Wybierz odpowiednio „silny” algorytm klasteryzacji. Wykorzystaj Clustering — scikit-learn 1.2.2 documentation i zamieszczony na Teams wykład. Możesz posłużyć się podziałem grafu najbliższych sąsiadów (graph cut and partition).
2. Na odgadniętych etykietach (i tych wcześniej znanych – zbiór treningowy) naucz sieć (także jej architektura może być dowolna) rozpoznawać klasy.
3. Dane bez odgadniętych etykiet użyj jako zbiór testowy. 
4. Powtórz 1-3 kilka razy.

### import bibliotek

In [1]:
import numpy as np
from collections import Counter
from sklearn.datasets import load_iris, load_wine
from sklearn.cluster import SpectralClustering
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings

# Wyłączamy ostrzeżenia, aby notebook był czytelniejszy
warnings.filterwarnings("ignore")

 ### Implementacja Pojedynczego Eksperymentu Pół-nadzorowanego

In [2]:
def run_semi_supervised(X, y, C=3, label_pct=0.05, seed=0):
    """
    Przeprowadza pojedynczy eksperyment uczenia pół-nadzorowanego:
    1. Losuje 'label_pct'% oznaczonych przykładów.
    2. Dzieli pozostałe dane na zbiory 'guess' i 'test'.
    3. Etykietuje klastry na podstawie głosowania większości znanego punktu.
    4. Trenuje klasyfikator MLP i zwraca dokładność na zbiorze testowym.
    """
    rng = np.random.default_rng(seed)
    n = X.shape[0]
    n_lab = int(label_pct * n)

    idx_all = np.arange(n)
    idx_lab = rng.choice(idx_all, n_lab, replace=False)
    idx_unlab = np.setdiff1d(idx_all, idx_lab)

    rng.shuffle(idx_unlab)
    split = len(idx_unlab) // 2
    idx_guess, idx_test = idx_unlab[:split], idx_unlab[split:]

    # Klasteryzacja na całym zbiorze X
    clust = SpectralClustering(
        n_clusters=C, affinity="nearest_neighbors",
        n_neighbors=10, random_state=seed
    )
    clusters = clust.fit_predict(X)

    # Mapowanie klaster -> klasa na podstawie znanych etykiet (majority vote)
    cl2class = {}
    for c in range(C):
        members = np.where(clusters == c)[0]
        known = np.intersect1d(members, idx_lab)
        cl2class[c] = (
            Counter(y[known]).most_common(1)[0][0] if known.size else -1
        )

    # Budowanie pseudo-etykiet dla unlabeled_guess_idx
    y_pseudo = np.ones_like(y) * -1 # Domyślnie 'brak etykiety'
    for i in idx_guess:
        cluster_label = clusters[i]
        if cl2class[cluster_label] != -1:
            y_pseudo[i] = cl2class[cluster_label]

    # Przygotowanie zbiorów treningowych i testowych
    train_mask = np.isin(idx_all, idx_lab) | (
        np.isin(idx_all, idx_guess) & (y_pseudo != -1)
    )
    
    X_train = X[train_mask]
    # Jeśli y_pseudo jest -1, używamy y_true (dla oryginalnych 5% etykiet), w przeciwnym razie pseudo-etykietę
    y_train = np.where(y_pseudo[train_mask] != -1, y_pseudo[train_mask], y[train_mask])

    X_test = X[idx_test]
    y_test = y[idx_test]

    # Trenowanie klasyfikatora MLP
    clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=seed)
    clf.fit(X_train, y_train)

    acc = accuracy_score(y_test, clf.predict(X_test))
    return acc

### Funkcja do Wielokrotnych Przebiegów i Wariacji 

In [3]:
def sweep_C_and_repeats(X, y, C_list=(2, 3, 4, 5), repeats=10):
    """
    Przeprowadza wielokrotne eksperymenty dla różnych wartości C (liczby klastrów)
    i zwraca średnią dokładność oraz odchylenie standardowe.
    """
    results = {C: [] for C in C_list}
    for C in C_list:
        for rep in range(repeats):
            acc = run_semi_supervised(X, y, C=C, seed=rep)
            results[C].append(acc)
    return {C: (np.mean(a), np.std(a)) for C, a in results.items()}

### Przeprowadzenie Eksperymentów dla Iris i Wine (Pół-nadzorowane)

In [4]:
# Wczytanie danych
X_iris, y_iris = load_iris(return_X_y=True)
X_wine, y_wine = load_wine(return_X_y=True)

# Uruchomienie eksperymentów pół-nadzorowanych
iris_res = sweep_C_and_repeats(X_iris, y_iris)
wine_res = sweep_C_and_repeats(X_wine, y_wine)

print("Wyniki dla zbioru Iris (Pół-nadzorowane):", iris_res)
print("Wyniki dla zbioru Wine (Pół-nadzorowane):", wine_res)

Wyniki dla zbioru Iris (Pół-nadzorowane): {2: (np.float64(0.5527777777777778), np.float64(0.18308063392537868)), 3: (np.float64(0.7625), np.float64(0.21113395344258654)), 4: (np.float64(0.7236111111111111), np.float64(0.2027444797622714)), 5: (np.float64(0.725), np.float64(0.20134578082689025))}
Wyniki dla zbioru Wine (Pół-nadzorowane): {2: (np.float64(0.3823529411764706), np.float64(0.14525958151103577)), 3: (np.float64(0.36235294117647054), np.float64(0.10172219445278721)), 4: (np.float64(0.36941176470588233), np.float64(0.12194502136329353)), 5: (np.float64(0.38235294117647056), np.float64(0.12954538556203082))}


### Implementacja Linii Bazowej (W pełni nadzorowane z 5% danych)

In [5]:
from sklearn.model_selection import train_test_split

def baseline_supervised(X, y, label_pct=0.05, seed=0):
    """
    Uczenie w pełni nadzorowane, wykorzystujące tylko 'label_pct'% danych.
    Służy jako punkt odniesienia.
    """
    # Dzielimy dane na zbiory treningowy i testowy. Zbiór treningowy ma tylko label_pct% danych.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=1 - label_pct, stratify=y, random_state=seed
    )

    clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=seed)
    clf.fit(X_train, y_train)

    return accuracy_score(y_test, clf.predict(X_test))

### Funkcja Podsumowująca Wyniki

In [6]:
def summarize_results(semi_supervised_results, full_supervised_results):
    """
    Generuje tabelę podsumowującą wyniki dla trybu pół-nadzorowanego (najlepsze C)
    i trybu w pełni nadzorowanego (dla 5% danych).
    """
    rows = []

    # Wyniki dla trybu pół-nadzorowanego
    best_C = max(semi_supervised_results, key=lambda c: semi_supervised_results[c][0])
    rows.append([
        "Pół-nadzorowane (najlepsze C)", best_C,
        *map(lambda x: round(x, 3), semi_supervised_results[best_C])
    ])

    # Wyniki dla trybu w pełni nadzorowanego
    rows.append([
        "W pełni nadzorowane (5%)", "-",
        round(np.mean(full_supervised_results), 3), round(np.std(full_supervised_results), 3)
    ])
    
    return rows

### Obliczenie Linii Bazowych i Wyświetlenie Wyników

In [9]:
import pandas as pd # Dodaj ten import na początku notebooka, jeśli jeszcze go nie ma

# Obliczenie wyników dla linii bazowej (w pełni nadzorowane z 5% danych)
iris_baseline = [baseline_supervised(X_iris, y_iris, seed=s) for s in range(10)]
wine_baseline = [baseline_supervised(X_wine, y_wine, seed=s) for s in range(10)]

# Podsumowanie i przygotowanie danych dla tabel
iris_table_data = summarize_results(iris_res, iris_baseline)
wine_table_data = summarize_results(wine_res, wine_baseline)

# Definicja nagłówków kolumn
headers = ["Tryb", "C", "Średnia dokładność", "Odchylenie std"]

# Tworzenie i wyświetlanie tabel dla Iris
print("\n Wyniki dla zbioru: Iris")
df_iris = pd.DataFrame(iris_table_data, columns=headers)
print(df_iris.to_markdown(index=False)) # Używamy to_markdown dla łatwego kopiowania i wklejania w markdown

# Tworzenie i wyświetlanie tabel dla Wine
print("\n Wyniki dla zbioru: Wine")
df_wine = pd.DataFrame(wine_table_data, columns=headers)
print(df_wine.to_markdown(index=False))


 Wyniki dla zbioru: Iris
| Tryb                          | C   |   Średnia dokładność |   Odchylenie std |
|:------------------------------|:----|---------------------:|-----------------:|
| Pół-nadzorowane (najlepsze C) | 3   |                0.762 |            0.211 |
| W pełni nadzorowane (5%)      | -   |                0.901 |            0.052 |

 Wyniki dla zbioru: Wine
| Tryb                          | C   |   Średnia dokładność |   Odchylenie std |
|:------------------------------|:----|---------------------:|-----------------:|
| Pół-nadzorowane (najlepsze C) | 2   |                0.382 |            0.145 |
| W pełni nadzorowane (5%)      | -   |                0.388 |            0.126 |


### analiza wyników

**Konfiguracja MLP:**

- Warstwa ukryta: 50 neuronów z funkcją aktywacji ReLU.
- Optymalizator: Adam, współczynnik uczenia lr=0.001.
- Maksymalna liczba iteracji: 300.
- Funkcja straty: entropia krzyżowa.


**Zbiór danych Iris:**
* Najlepszą wydajność w scenariuszu pół-nadzorowanym osiągnęliśmy dla $C=3$, co jest zgodne z rzeczywistą liczbą klas w zbiorze Iris.
* Obserwujemy znaczną różnicę w dokładności (~14 punktów procentowych) w porównaniu do modelu trenowanego wyłącznie na prawdziwych 5% etykietowanych danych.
* Duże odchylenie standardowe ($\sigma \approx 0.21$) wskazuje na brak stabilności. Oznacza to, że wyniki są bardzo wrażliwe na konkretny wybór początkowych 5% etykietowanych próbek oraz na sposób, w jaki klasteryzacja Spectral Clustering dzieli dane.
* Wniosek: Jakość generowanych pseudo-etykiet jest niejednolita; ponieważ Iris jest małym zbiorem danych, każdy błąd w przypisaniu etykiety ma duży wpływ na końcową dokładność.

**Zbiór danych Wine:**
* Co zaskakujące, najwyższą dokładność w trybie pół-nadzorowanym uzyskaliśmy dla $C=2$ , co sugeruje, że Spectral Clustering miał trudności z identyfikacją trzech faktycznych typów wina.
* Tryb pół-nadzorowany praktycznie nie przynosi zysku w porównaniu do linii bazowej (0.382 vs 0.388); mamy szum, ale zero wartości dodanej.
* Podejrzewamy, że wysoka, 13-wymiarowa przestrzeń cech w połączeniu z niewielką liczbą ręcznie etykietowanych próbek skutkuje niską jakością macierzy sąsiedztwa, co negatywnie wpływa na klasteryzację i uniemożliwia prawidłowe rozpoznanie naturalnych klas.

**Ogólne wnioski:**
Przy 5% ręcznego etykietowania, **bardziej opłacalne jest często trenowanie sieci w trybie w pełni nadzorowanym** niż próba "doklejania" ryzykownych pseudo-etykiet.

### Potencjalne Ulepszenia:
* **Redukcja wymiarowości:** Zastosowanie technik takich jak PCA lub t-SNE przed klasteryzacją.
* **Zmiana algorytmu klasteryzacji:** Eksperymentowanie z innymi algorytmami, np.DBSCAN, HDBSCAN, lub dokładne dostrojenie parametrów k-NN dla Spectral Clustering.
* **Zwiększenie odsetka ręcznie etykietowanych próbek:** Podniesienie tej wartości (np. do 10%) może znacząco poprawić jakość klastrów i stabilność modelu.